In [1]:
!pip install pyreadstat

   ---------------------------------------- 0.0/2.4 MB ? eta -:--:--
   -------- ------------------------------- 0.5/2.4 MB 5.6 MB/s eta 0:00:01
   -------- ------------------------------- 0.5/2.4 MB 5.6 MB/s eta 0:00:01
   -------- ------------------------------- 0.5/2.4 MB 5.6 MB/s eta 0:00:01
   -------- ------------------------------- 0.5/2.4 MB 5.6 MB/s eta 0:00:01
   ----------------- ---------------------- 1.0/2.4 MB 915.5 kB/s eta 0:00:02
   --------------------- ------------------ 1.3/2.4 MB 1.1 MB/s eta 0:00:02
   --------------------- ------------------ 1.3/2.4 MB 1.1 MB/s eta 0:00:02
   --------------------- ------------------ 1.3/2.4 MB 1.1 MB/s eta 0:00:02
   --------------------- ------------------ 1.3/2.4 MB 1.1 MB/s eta 0:00:02
   ---------------------------------- ----- 2.1/2.4 MB 931.9 kB/s eta 0:00:01
   ---------------------------------- ----- 2.1/2.4 MB 931.9 kB/s eta 0:00:01
   ---------------------------------- ----- 2.1/2.4 MB 931.9 kB/s eta 0:00:01
   --------

In [36]:
# Check the contents of the 'Dataset' folder inside the extracted path
import os
dataset_folder = os.path.dirname('C:/DEPI_Project/Dataset/')
os.listdir(dataset_folder)


['AGP_L.xpt',
 'ALQ_L.xpt',
 'BAQ_L.xpt',
 'BAX_L.xpt',
 'BMX_L.xpt',
 'BPQ_L.xpt',
 'BPXO_L.xpt',
 'CBC_L.xpt',
 'DBQ_L.xpt',
 'DEMO_L.xpt',
 'DIQ_L.xpt',
 'DSQTOT_L.xpt',
 'ECQ_L.xpt',
 'FERTIN_L.xpt',
 'FOLATE_L.xpt',
 'FOLFMS_L.xpt',
 'GHB_L.xpt',
 'GLU_L.xpt',
 'HDL_L.xpt',
 'HSCRP_L.xpt',
 'HSQ_L.xpt',
 'IHGEM_L.xpt',
 'INS_L (1).xpt',
 'LUX_L.xpt',
 'OHQ_L.xpt',
 'PAQ_L.xpt',
 'PBCD_L.xpt',
 'RHQ_L.xpt',
 'RXQASA_L.xpt',
 'SLQ_L.xpt',
 'SMQ_L.xpt',
 'TCHOL_L.xpt',
 'VID_L.xpt',
 'WHQ_L.xpt']

In [ ]:
import pandas as pd
from pathlib import Path
import pyreadstat
from collections import defaultdict

column_sources = defaultdict(list)
merged_df = None
# Load all XPT files into dataframes (except OHQ_L.xpt which is for oral health)
dataframes = {}
for file in os.listdir(dataset_folder):
    if file.endswith('.xpt') :
        file_path = os.path.join(dataset_folder, file)
        df, meta = pyreadstat.read_xport(file_path, encoding='cp1252')
        key = Path(file).stem
        try:
            df, meta = pyreadstat.read_xport(file_path, encoding='cp1252')
            dataframes[key] = df
            print(f"✅ Loaded: {file}")
        except Exception as e:
            print(f"❌ Failed to load {file}: {e}")
        print(key)
        print(dataframes[key].shape)
        
        for var, label in zip(meta.column_names, meta.column_labels):
            print(f"{var}: {label}")
            #pass
        if merged_df is None:
            merged_df = df
        else:
            # Only keep columns that are not already in merged_df (except 'SEQN')
            df_filtered = df.loc[:, ~df.columns.isin(merged_df.columns.difference(['SEQN']))]
            merged_df = pd.merge(merged_df, df_filtered, on="SEQN", how="outer")
        for col in df.columns:
            column_sources[col].append(file)

# Find duplicates (i.e., columns that appear in more than one file)
duplicate_columns = {col: sources for col, sources in column_sources.items() if len(sources) > 1}

# Show them
for col, sources in duplicate_columns.items():
    print(f"🔁 Column '{col}' appears in: {sources}")
# Show the loaded dataset keys
#list(dataframes.keys())
merged_df.shape, merged_df.head()


✅ Loaded: AGP_L.xpt
AGP_L
(2564, 3)
SEQN: Respondent sequence number
WTPH2YR: Phlebotomy 2 Year Weight
LBXAGP: alpha-1-acid glycoprotein (g/L)
✅ Loaded: ALQ_L.xpt
ALQ_L
(6337, 9)
SEQN: Respondent sequence number
ALQ111: Ever had a drink of any kind of alcohol
ALQ121: Past 12 mos how often drink alc bev
ALQ130: Avg # alcoholic drinks/day/past 12 mos
ALQ142: # days have 4/5 drinks/past 12 mos
ALQ270: # times 4/5 drinks in 2hrs/past 12 mos
ALQ280: # times 8+ drinks in 1 day/past 12 mos
ALQ151: Ever have 4/5 or more drinks every day
ALQ170: # times 4/5 drinks on occasion/past mo
✅ Loaded: BAQ_L.xpt
BAQ_L
(6070, 15)
SEQN: Respondent sequence number
BAQ321A: Past 12 months, problems with vertigo
BAQ321B: Past 12 months, prblms w/blurring vision
BAQ321C: Past 12 months, problems with unsteady
BAQ321D: Past 12 months, problems w/light-headed
BAQ341: Past 12 months, which bothered the most?
BAQ391A: Episodes with nausea or vomiting?
BAQ391B: Episodes with migraine/severe headache?
BAQ401: Balan

((11933, 355),
        SEQN       WTPH2YR  LBXAGP  ALQ111  ALQ121  ALQ130  ALQ142  ALQ270  \
 0  130378.0           NaN     NaN     NaN     NaN     NaN     NaN     NaN   
 1  130379.0           NaN     NaN     1.0     2.0     3.0     0.0     NaN   
 2  130380.0  85328.844519   1.010     1.0    10.0     1.0     0.0     NaN   
 3  130381.0      0.000000     NaN     NaN     NaN     NaN     NaN     NaN   
 4  130382.0  59638.932323   0.921     NaN     NaN     NaN     NaN     NaN   
 
    ALQ280  ALQ151  ...  LBXVD2MS  LBDVD2LC  LBXVD3MS  LBDVD3LC  LBXVE3MS  \
 0     NaN     NaN  ...      1.57       1.0      57.3       0.0      2.47   
 1     NaN     2.0  ...      1.57       1.0      58.9       0.0      3.40   
 2     NaN     2.0  ...      1.57       1.0      37.8       0.0      1.52   
 3     NaN     NaN  ...       NaN       NaN       NaN       NaN       NaN   
 4     NaN     NaN  ...       NaN       NaN       NaN       NaN       NaN   
 
    LBDVE3LC  WHD010  WHD020  WHD050  WHQ070  
 0  

In [ ]:
merged_df.to_csv("merged_output.csv", index=False)


In [3]:
import pandas as pd
from pathlib import Path
import pyreadstat
from collections import defaultdict
import numpy as np
new_df=pd.read_csv("C:/DEPI_Project/merged_output.csv")

In [4]:
display(new_df.info(),new_df.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11933 entries, 0 to 11932
Columns: 355 entries, SEQN to WHQ070
dtypes: float64(347), object(8)
memory usage: 32.3+ MB


None

,SEQN,WTPH2YR,LBXAGP,ALQ111,ALQ121,ALQ130,ALQ142,ALQ270,ALQ280,ALQ151,...,LBXVD2MS,LBDVD2LC,LBXVD3MS,LBDVD3LC,LBXVE3MS,LBDVE3LC,WHD010,WHD020,WHD050,WHQ070
0,130378.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.57,1.0,57.3,0.0,2.47,0.0,71.0,190.0,200.0,1.0
1,130379.0,NaN,NaN,1.0,2.0,3.0,0.0,NaN,NaN,2.0,...,1.57,1.0,58.9,0.0,3.40,0.0,70.0,220.0,220.0,2.0
2,130380.0,85328.844519,1.010,1.0,10.0,1.0,0.0,NaN,NaN,2.0,...,1.57,1.0,37.8,0.0,1.52,0.0,60.0,150.0,165.0,1.0
3,130381.0,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,130382.0,59638.932323,0.921,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
new_df.describe().T

,count,mean,std,min,25%,50%,75%,max
SEQN,11933.0,136344.000000,3444.904716,130378.000,133361.000000,136344.000000,139327.000000,142310.000000
WTPH2YR,2564.0,38852.911512,35557.669012,0.000,15233.708494,32116.487072,54707.666378,253478.777645
LBXAGP,1950.0,0.775883,0.250911,0.261,0.588000,0.751500,0.922750,2.760000
ALQ111,5481.0,1.109104,0.385114,1.000,1.000000,1.000000,1.000000,9.000000
ALQ121,4922.0,5.030679,4.314321,0.000,2.000000,5.000000,8.000000,99.000000
...,...,...,...,...,...,...,...,...
LBDVE3LC,7238.0,0.009671,0.097872,0.000,0.000000,0.000000,0.000000,1.000000
WHD010,8486.0,150.146712,906.045672,40.000,63.000000,66.000000,70.000000,9999.000000
WHD020,8486.0,317.142588,1120.476550,63.000,145.000000,174.000000,207.000000,9999.000000
WHD050,8486.0,420.465237,1491.928102,67.000,145.000000,175.000000,214.000000,9999.000000


In [ ]:
##Detect Null Values
#new_df.isnull().sum()
print(new_df.isnull().sum().to_string())


SEQN            0
WTPH2YR      9369
LBXAGP       9983
ALQ111       6452
ALQ121       7011
ALQ130       7864
ALQ142       7851
ALQ270       9567
ALQ280       9571
ALQ151       7032
ALQ170       9575
BAQ321A      5874
BAQ321B      5874
BAQ321C      5875
BAQ321D      5875
BAQ341       9794
BAQ391A      9811
BAQ391B      9811
BAQ401       9794
BAQ421       9794
BAQ431       9794
BAQ491       9794
BAQ530       5873
BAQ550       9123
BAQ560      10210
BAXMSTAT     7162
BAXRXNC     11729
BAXRXND     11720
BAX5STAT     7368
BAQ110       7570
BAQ121       7628
BAQ125       7654
BAQ132       7909
BAQ140       7909
BAQ150       7961
BAQ160       8315
BAQ170       8418
BAQ201       8646
BAQ173       7965
BAXPF11      7982
BAXTC11      7983
BAARFC11    11884
BAXPF12     11885
BAXTC12     11885
BAARFC12    11928
BAXPF21      7990
BAXTC21      7991
BAARFC21    11839
BAXPF22     11836
BAXTC22     11839
BAARFC22    11907
BAXPF31      8036
BAXTC31      8051
BAARFC31    11853
BAXPF32     11853
BAXTC32   

In [22]:
missing_over_50 = new_df.isnull().mean() > 0.5
columns_with_missing_over_50 = new_df.columns[missing_over_50]

# To see the counts of missing values in those columns:
missing_counts = new_df[columns_with_missing_over_50].isnull().sum()
pd.set_option('display.max_rows', None)
print(missing_counts)


WTPH2YR      9369
LBXAGP       9983
ALQ111       6452
ALQ121       7011
ALQ130       7864
ALQ142       7851
ALQ270       9567
ALQ280       9571
ALQ151       7032
ALQ170       9575
BAQ341       9794
BAQ391A      9811
BAQ391B      9811
BAQ401       9794
BAQ421       9794
BAQ431       9794
BAQ491       9794
BAQ550       9123
BAQ560      10210
BAXMSTAT     7162
BAXRXNC     11729
BAXRXND     11720
BAX5STAT     7368
BAQ110       7570
BAQ121       7628
BAQ125       7654
BAQ132       7909
BAQ140       7909
BAQ150       7961
BAQ160       8315
BAQ170       8418
BAQ201       8646
BAQ173       7965
BAXPF11      7982
BAXTC11      7983
BAARFC11    11884
BAXPF12     11885
BAXTC12     11885
BAARFC12    11928
BAXPF21      7990
BAXTC21      7991
BAARFC21    11839
BAXPF22     11836
BAXTC22     11839
BAARFC22    11907
BAXPF31      8036
BAXTC31      8051
BAARFC31    11853
BAXPF32     11853
BAXTC32     11854
BAARFC32    11913
BAXPF41      8070
BAXTC41      8083
BAARFC41    10543
BAXPF42     10547
BAXTC42   

In [7]:
print(new_df['DIQ010'].value_counts())

'''Variable Name: DIQ010
Target: "Doctor told you have diabetes?"Values:
  1 = Yes
  2 = No
  3 = Borderline
  7 = Refused
  9 = Don’t know'''

DIQ010
2.0    10371
1.0     1081
3.0      284
9.0        4
Name: count, dtype: int64


'Variable Name: DIQ010\nTarget: "Doctor told you have diabetes?"Values:\n  1 = Yes\n  2 = No\n  3 = Borderline\n  7 = Refused\n  9 = Don’t know'

In [8]:
import plotly.offline as py
import plotly.graph_objs as go
# 2 datasets
D = new_df[(new_df['DIQ010'] == 1)]
H = new_df[(new_df['DIQ010'] == 2)]

#------------COUNT-----------------------
def target_count():
    trace = go.Bar( x = new_df['DIQ010'].value_counts().values.tolist(),
                    y = ['healthy','diabetic' ],
                    orientation = 'h',
                    text=new_df['DIQ010'].value_counts().values.tolist(),
                    textfont=dict(size=15),
                    textposition = 'auto',
                    opacity = 0.8,marker=dict(
                    color=['lightskyblue', 'gold'],
                    line=dict(color='#000000',width=1.5)))

    layout = dict(title =  'Count of Outcome variable')

    fig = dict(data = [trace], layout=layout)
    py.iplot(fig)

#------------PERCENTAGE-------------------
def target_percent():
    trace = go.Pie(labels = ['healthy','diabetic'], values = new_df['DIQ010'].value_counts(),
                   textfont=dict(size=15), opacity = 0.8,
                   marker=dict(colors=['lightskyblue', 'gold'],
                               line=dict(color='#000000', width=1.5)))


    layout = dict(title =  'Distribution of Outcome variable')

    fig = dict(data = [trace], layout=layout)
    py.iplot(fig)

In [9]:
target_count()
target_percent()

In [15]:
###Correlation
import numpy as np
def correlation_plot():
    #correlation
    numeric_df=new_df.select_dtypes(include='number')
    correlation = numeric_df.corr()
    #tick labels
    matrix_cols = correlation.columns.tolist()
    #convert to array
    corr_array  = np.array(correlation)
    trace = go.Heatmap(z = corr_array,
                       x = matrix_cols,
                       y = matrix_cols,
                       colorscale='Viridis',
                       colorbar   = dict() ,
                      )
    layout = go.Layout(dict(title = 'Correlation Matrix for variables',
                            autosize = False,
                            height  = 5000,
                            width   = 6000,
                            margin  = dict(r = 0 ,l = 100,
                                           t = 0,b = 100,
                                         ),
                            yaxis   = dict(tickfont = dict(size = 9)),
                            xaxis   = dict(tickfont = dict(size = 9)),
                           )
                      )
    fig = go.Figure(data = [trace],layout = layout)
    py.iplot(fig)

In [16]:
correlation_plot()